In [1]:
def lookupUserById():
    userProfile = myApi.lookup_users(user_ids=['813286'])
    
    for user in userProfile:
        print ("Twitter Handle = ", user.screen_name)
        print ("Username = ",user.name)
        print ("Description = ",user.description)
        print ("User ID = ", user.id)
        print ("Total Followers = ",user.followers_count)
        print ("Total Tweets = ",user.statuses_count)
        print ("Web URL = ",user.url, '\n')
        
        #user profile description
        terms_stop = [term for term in preprocess(user.description) if term not in stop]
        # Update the counter
        count_all.update(terms_stop)

        # Print the first N most frequent words
        print("Frequent words: ", count_all.most_common(20))

def tokenize(s):
    return tokens_re.findall(s)

    #args s = tweets, lowercase=False(keeps as in tweets) / True(changes all to lowercase)
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tweepy
import csv
import re
import string
import unicodedata

#Suppress warnings
import warnings
warnings.filterwarnings('ignore')

from IPython import display
from nltk.corpus import stopwords
from collections import Counter
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score, train_test_split


#Auth details from Twitter app http://twitclassify.com/
CONSUMER_KEY = "90bt0hlKzf3zWi5aPJ5ELZOGL"
CONSUMER_SECRET = "H4ycskJVRjcesIaZBVUFdTClsucdMEyPcZyQT7Xjvou3LWwBg2"
ACCESS_KEY = "303590178-hw3mgzNW08xAvbrVW4Zm8VP4uKpU9qyfG01ynkzi"
ACCESS_SECRET = "6wJw3hGpFAgR77nk6Hw4UjCGi96OtOZhnXjm1iMcptQ3i"

authid = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
authid.set_access_token(ACCESS_KEY, ACCESS_SECRET)
myApi = tweepy.API(authid)


punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'if', 'i', '’', '“', '”', 'it', 'the', '…', 'us', ':)', '—',
                                                  ':/', 'il', '']

# Source taken from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' #anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

data_train = np.genfromtxt("trainingmix.tsv", delimiter='\t', dtype= str)

features = []
li=[]
ascii=[]
twitHandle = data_train[:,1] # this gets the tweet handles [column B]

#print(twitHandle)
#print(y)

count_all = Counter()

for i in range(len(twitHandle)):
    display.clear_output(wait=True) #refresh display
    print(i+1) # Show progress of user retrieval
    #get user's timeline of recent tweets on # of items
    for status in tweepy.Cursor(myApi.user_timeline, screen_name = twitHandle[i]).items(5):

        
        terms_stop = [term for term in preprocess(status.text) if term not in stop]
            
        # Source taken from https://marcobonzanini.com/2015/03/17/mining-twitter-data-with-python-part-3-term-frequencies/    
        # Count hashtags only
        terms_hash = [term for term in preprocess(status.text) 
                  if term.startswith('#')]
        # Count terms only (no hashtags, no mentions)
        terms_only = [term for term in preprocess(status.text) 
                  if term not in stop and
                  not term.startswith(('#', '@', 'http'))] 
    
        #print("user id :", status.user.id)

        # Update the counter
        count_all.update(terms_only)
        features = str(count_all.most_common(8)) # N must be even number
        features = [term for term in preprocess(features) if term not in stop]
        features = [term.encode('unicode_escape').decode('UTF-8') for term in features]
        
    #ascii = [[ord(ch) for ch in word] for word in features]
    for word in features:
        innerlist = []
        for ch in word:
            innerlist.append(ord(ch)) # ','.join somwehere here
        ascii.append(innerlist)
    
    li = ascii[0::2] #obtain all words, omitting counters
    #print (li)
    del features[:] # Empty List for next user most freq words
    count_all.clear() # Clear counter
    
    spamWriter = csv.writer(open('Trainingshttt.csv', 'a', newline=''), delimiter=',')
    spamWriter.writerow(li)

print("Done!")

500
Done!


In [ ]:
#SVM Classifier full data
# Cross Validation to fit data and get accuracy
# find best value of C

scoreSVM=[]
accuracy=[]

for i in range (1,11):
    modelSVM = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, tol=0.0001, C=i, multi_class='ovr',
                     fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                     random_state=None, max_iter=1000)
    scoreMean = (cross_val_score(modelSVM, trainX, train_y, cv=2)).mean()
    scoreSVM.append(scoreMean)
    modelSVM.fit(trainX, train_y)
    expected = test_y
    predicted = modelSVM.predict(testX)
    accuracy.append(metrics.accuracy_score(expected, predicted))

for i in range (0,10):
    print("Score Mean = ", scoreSVM[i])

plt.xlabel('Value of C')
plt.ylabel('Accuracy')
plt.title('Values of C against Accuracy Score')

for i in range (0,10):
    plt.plot(i+1,accuracy[i],'ro')

plt.show()
    
# fit a SVM model to the data
modelSVM = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, tol=0.0001, C=5.0, multi_class='ovr',
                     fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                     random_state=None, max_iter=1000)
modelSVM.fit(trainX, train_y)
print(modelSVM)
# make predictions
expected = test_y
predicted = modelSVM.predict(testX)
# summarize the fit of the model
# Classification report
print("Classification report ", metrics.classification_report(expected, predicted))

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = ", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)